In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [29]:
# df = pd.read_excel('../data/TFBU_MPRA/supp/41467_2025_56749_MOESM11_ESM.xlsx')
# df = df.rename(columns={'enhancer sequence': 'seq', 'measured enhancer activity': 'HepG2', 'sequence_name': 'id'})
# df = df[['seq', 'id', 'HepG2']]
# df.to_csv('../data/TFBU_MPRA/raw/HepG2_mean.tsv', index=False, sep='\t')

In [30]:
cell_types = ['HepG2', 'HEK293T', 'Jurkat', 'PANC-1', 'HeLa-S3']

In [31]:
for i, cell_type in enumerate(cell_types):
    df1 = pd.read_csv(f'../data/TFBU_MPRA/raw/7_exp_{cell_type}_rep1.tsv', sep='\t', header=None, names=['seq', f'{cell_type}_1', 'id'])
    df2 = pd.read_csv(f'../data/TFBU_MPRA/raw/7_exp_{cell_type}_rep2.tsv', sep='\t', header=None, names=['seq', f'{cell_type}_2', 'id'])
    df3 = pd.read_csv(f'../data/TFBU_MPRA/raw/7_exp_{cell_type}_rep3.tsv', sep='\t', header=None, names=['seq', f'{cell_type}_3', 'id'])
    df_merged = pd.DataFrame(columns=["seq", "id"])
    df_merged = pd.merge(df_merged, df1, on=['seq', 'id'], how="outer")
    df_merged = pd.merge(df_merged, df2, on=['seq', 'id'], how="outer")
    df_merged = pd.merge(df_merged, df3, on=['seq', 'id'], how="outer")
    df_merged[cell_type] = np.nanmean(np.log2(df_merged[[f'{cell_type}_1', f'{cell_type}_2', f'{cell_type}_3']]), axis=1)
    # df_merged[cell_type] = np.log2(np.nanmean(df_merged[[f'{cell_type}_1', f'{cell_type}_2', f'{cell_type}_3']], axis=1))
    df_merged = df_merged.drop([f'{cell_type}_1', f'{cell_type}_2', f'{cell_type}_3'], axis=1)
    df_merged.to_csv(f'../data/TFBU_MPRA/raw/{cell_type}_log_mean.tsv', sep='\t', index=False)

In [32]:
from Bio import SeqIO

def fasta_to_df(fasta_path):
    records = [
        {"id": record.id, "seq": str(record.seq)}
        for record in SeqIO.parse(fasta_path, "fasta")
    ]
    return pd.DataFrame(records)

df = fasta_to_df("../data/TFBU_MPRA/raw/library1_final_edit50-0509.fasta")
df

,id,seq
0,0.7488720799999999_ARNT2_pos_00_chr17_43361125...,ACTGGCCGCTTGACGCGGATTTAAGCGTGTAAACGCAACACACAAC...
1,0.7244516230000001_ARNT2_pos_00_chr3_190424655...,ACTGGCCGCTTGACGTCGCCTCCCTCAGACTCTGCACCCGCCACAA...
2,0.697715514_ARNT2_pos_00_chr20_46174909_461750...,ACTGGCCGCTTGACGCGAAGTCGTAGAGGCTCCGCAGCGCCGACAT...
3,0.6746311209999999_ARNT2_pos_00_chr20_31968460...,ACTGGCCGCTTGACGTTTGGTCCGGAAGGCGCCGGCGCTGTCCTTG...
4,0.664452299_ARNT2_pos_00_chr22_41666645_416668...,ACTGGCCGCTTGACGCGCGATGGCCGCGGTAGAAGTCAAGCGCCAG...
...,...,...
17995,verified_sequence_51,ACTGGCCGCTTGACGCCGAAGTTGGCGCCTGCGGTGCGGCCGCCGC...
17996,verified_sequence_52,ACTGGCCGCTTGACGCGGCGCCGGCGCGGCGTGGTACCGTAAGCCG...
17997,verified_sequence_53,ACTGGCCGCTTGACGTACTTGAGCCCAGGAGTTGAGACCAGCCTAA...
17998,verified_sequence_54,ACTGGCCGCTTGACGCCCTACACACATACAGCCTCCCTCATTATCA...


In [33]:
# df['left_adapter'] = df['seq'].str.slice(0, 15)
# df['right_adapter'] = df['seq'].str.slice(-15, None)
df['seq'] = df['seq'].str.slice(15, -15)
df

,id,seq
0,0.7488720799999999_ARNT2_pos_00_chr17_43361125...,CGGATTTAAGCGTGTAAACGCAACACACAACCTAGGAGGTCAGTAA...
1,0.7244516230000001_ARNT2_pos_00_chr3_190424655...,TCGCCTCCCTCAGACTCTGCACCCGCCACAACGTCTGGCTAACTTT...
2,0.697715514_ARNT2_pos_00_chr20_46174909_461750...,CGAAGTCGTAGAGGCTCCGCAGCGCCGACATGTCGTAGGCTTCGGT...
3,0.6746311209999999_ARNT2_pos_00_chr20_31968460...,TTTGGTCCGGAAGGCGCCGGCGCTGTCCTTGGTGCTGATGGCGGCT...
4,0.664452299_ARNT2_pos_00_chr22_41666645_416668...,CGCGATGGCCGCGGTAGAAGTCAAGCGCCAGAGGGCACTCCATGCG...
...,...,...
17995,verified_sequence_51,CCGAAGTTGGCGCCTGCGGTGCGGCCGCCGCTTGCCGCCGAATACG...
17996,verified_sequence_52,CGGCGCCGGCGCGGCGTGGTACCGTAAGCCGATTCCGAATCGCGGA...
17997,verified_sequence_53,TACTTGAGCCCAGGAGTTGAGACCAGCCTAAGCAATATAGTGAGAC...
17998,verified_sequence_54,CCCTACACACATACAGCCTCCCTCATTATCAGCATTCCCCACCAGA...


In [34]:
dfs = [
    pd.read_csv(f'../data/TFBU_MPRA/raw/{ct}_log_mean.tsv', sep='\t')
    for ct in cell_types
]

for ct, df_ in zip(cell_types, dfs):
    print(ct, df_.shape)


for df_ in dfs:
    df = df.merge(df_, on=["seq", "id"], how="left")

df.describe()

HepG2 (16911, 3)
HEK293T (16942, 3)
Jurkat (16106, 3)
PANC-1 (16949, 3)
HeLa-S3 (16795, 3)


,HepG2,HEK293T,Jurkat,PANC-1,HeLa-S3
count,16911.000,16942.000,16106.000,16949.000,16795.000
mean,-0.732,-1.172,-0.809,-0.878,-0.645
std,1.157,1.264,1.191,0.934,0.782
min,-2.586,-3.841,-4.431,-3.108,-3.414
25%,-1.564,-2.008,-1.580,-1.536,-1.121
50%,-1.146,-1.619,-1.086,-1.133,-0.857
75%,-0.230,-0.794,-0.297,-0.456,-0.414
max,5.756,8.135,4.978,9.229,9.763


In [35]:
df.to_csv('../data/TFBU_MPRA/TFBU_MPRA_18000.tsv', sep='\t', index=False)

In [9]:
MPRA_df = pd.read_csv('../data/TFBU_MPRA/TFBU_MPRA_18000.tsv', sep='\t')
MPRA_df

,id,seq,HepG2,HEK293T,Jurkat,PANC-1,HeLa-S3
0,0.7488720799999999_ARNT2_pos_00_chr17_43361125...,CGGATTTAAGCGTGTAAACGCAACACACAACCTAGGAGGTCAGTAA...,-0.520,-0.517,0.460,-0.583,-0.871
1,0.7244516230000001_ARNT2_pos_00_chr3_190424655...,TCGCCTCCCTCAGACTCTGCACCCGCCACAACGTCTGGCTAACTTT...,-0.651,-1.240,-1.899,-0.526,-0.428
2,0.697715514_ARNT2_pos_00_chr20_46174909_461750...,CGAAGTCGTAGAGGCTCCGCAGCGCCGACATGTCGTAGGCTTCGGT...,-0.744,0.013,-0.262,-0.675,-0.663
3,0.6746311209999999_ARNT2_pos_00_chr20_31968460...,TTTGGTCCGGAAGGCGCCGGCGCTGTCCTTGGTGCTGATGGCGGCT...,-1.642,-1.575,-1.819,-1.600,-0.599
4,0.664452299_ARNT2_pos_00_chr22_41666645_416668...,CGCGATGGCCGCGGTAGAAGTCAAGCGCCAGAGGGCACTCCATGCG...,0.099,-0.887,-0.789,-0.577,-0.952
...,...,...,...,...,...,...,...
17995,verified_sequence_51,CCGAAGTTGGCGCCTGCGGTGCGGCCGCCGCTTGCCGCCGAATACG...,-0.499,-0.666,-0.684,0.073,-0.202
17996,verified_sequence_52,CGGCGCCGGCGCGGCGTGGTACCGTAAGCCGATTCCGAATCGCGGA...,-0.569,-0.876,-0.942,0.286,-0.145
17997,verified_sequence_53,TACTTGAGCCCAGGAGTTGAGACCAGCCTAAGCAATATAGTGAGAC...,-0.313,-2.134,NaN,-1.650,-1.444
17998,verified_sequence_54,CCCTACACACATACAGCCTCCCTCATTATCAGCATTCCCCACCAGA...,-1.263,-2.604,-2.508,-1.932,-0.828


In [10]:
MPRA_df['seq_168bp'] = MPRA_df['seq']

In [11]:
left_seq = 'GTACGGGAGGTATTGGACAGGCCGCAATAAAATATCTTTATTTTCATTACATCTGTGTGTTGGTTTTTTGTGTGAATCGATAGTACTAACATACGCTCTCCATCAAAACAAAACGAAACAAAACAAACTAGCAAAATAGGCTGTCCCCAGTGCAAGTGCAGGTGCCAGAACATTTCTCTGGCCTAACTGGCCGCTTGACG'
right_seq = 'CACTGCGGCTCCTGCGATagagggtatataatggaagctcgacttccagcttggcaatccggtactgttggtaaagccaccatggtgagcaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtgtccggcgagggcgagggcgatgccaccta'
MPRA_df['seq'] = pad_seqs(MPRA_df['seq_168bp'], padded_length=200, padding_method='given', given_left_seq=left_seq, given_right_seq=right_seq)

In [13]:
MPRA_df.to_csv('../data/TFBU_MPRA/TFBU_MPRA_18000.tsv', sep='\t', index=False)